# Implement the Conjugate Gradient Method for Solving Linear Systems

## Task: Implement the Conjugate Gradient Method for Solving Linear Systems

Your task is to implement the Conjugate Gradient (CG) method, an efficient iterative algorithm for solving large, sparse, symmetric, positive-definite linear systems. Given a matrix A and a vector b, the algorithm will solve for x in the system Ax = b.

Write a function conjugate_gradient(A, b, n, x0=None, tol=1e-8) that performs the Conjugate Gradient method as follows:

- A: A symmetric, positive-definite matrix representing the linear system.
- b: The vector on the right side of the equation.
- n: Maximum number of iterations.
- x0: Initial guess for the solution vector.
- tol: Tolerance for stopping criteria.

The function should return the solution vector x.

Example
```python
import numpy as np

A = np.array([[4, 1], [1, 3]])
b = np.array([1, 2])
n = 5

print(conjugate_gradient(A, b, n))

# Expected Output:
# [0.09090909, 0.63636364]
```

## Understanding The Conjugate Gradient Method

The Conjugate Gradient, CG, method is an iterative algorithm used to solve large systems of linear equations, particularly those that are symmetric and positive-definite.

## Concepts

The CG gradient method is often applied to the quadratic form of a linear system, Ax = b:
 
$$f(x) = \frac{1}{2}x^TAx - b^Tx$$
 
The quadratic form is used due to its' differential reducing to, the following for a symmetric A. Therefore, x satisfies Ax = b, at the optimum:

$$\nabla f(x) = Ax - b = 0$$

The conjugate gradient method uses search directions that are conjugate to all the previous search direction. This is satisfied when search directions are A-orthogonal, i.e.

$$p_i^TAp_j = 0, \forall i \neq j$$

This results in a more efficient algorithm, as it ensures that the algorithm gathers all information in a search direction at once, and then doesn't need to search in that direction again. As opposed to steepest descent, that will step a bit in one direction and then may search in that direction later.

## Algorithm Steps

1. Initialization:
- $x_0$: Initial guess for the variable vector.
- $r_0=b-Ax_0$: Initial residual vector.
- $p_0=r_0$: Initial search direction.
2. Iteration k:
- $\alpha_k = \frac{r_k^Tr_k}{p_k^TAp_k}$: Step size.
- $x_{k+1} = x_k + \alpha_kp_k$: Update solution.
- $r_{k+1} = r_k - \alpha_kAp_k$: Update residual.
- Check convergence: $||r_{k+1}|| < \text{tolerance}$.
- $\beta_k = \frac{r_{k+1}^Tr_{k+1}}{r_k^Tr_k}$: New direction scaling. This ensures search directions are A-orthogonal.
- $p_{k+1} = r_{k+1} + \beta_kp_k$: Update search direction.
3. Termination:
- Stop when $||r_{k+1}|| < \text{tolerance}$ or after a set number of iterations.

## Example Calculation

Let's solve the system of equations:

$$4x_1 + x_2 = 6, x_1 + 3x_2 = 6$$

1. Initialize $x_0 = [0, 0]^T$, $r_0 = b - Ax_0 = [6, 6]^T$, $p_0 = r_0 = [6, 6]^T$.
2. First iteration:
- Compute $\alpha_0$:
  $$\alpha_0 = \frac{r_0^Tr_0}{p_0^TAp_0} = \frac{72}{324} = 0.2222$$
- Update solution $x_1$:
  $$x_1 = x_0 + \alpha_0p_0 = [0, 0]^T + 0.2222[6, 6]^T = [1.3333, 1.3333]^T$$
- Update residual $r_1$:
  $$r_1 = r_0 - \alpha_0Ap_0 = [6, 6]^T - 0.2222 \cdot \begin{bmatrix} 4 & 1 \\ 1 & 3 \end{bmatrix} \cdot [6, 6]^T = [6.67, 5.33]^T$$
- Compute $\beta_0$:
  $$\beta_0 = \frac{r_1^Tr_1}{r_0^Tr_0} = \frac{6.67^2 + 5.33^2}{6^2 + 6^2} \approx 0.99$$
- Update search direction $p_1$:
  $$p_1 = r_1 + \beta_0p_0 = [6.67, 5.33]^T + 0.99[6, 6]^T = [12.60, 11.26]^T$$
3. Second iteration:
- Compute $\alpha_1$, $x_2$, $r_2$, and repeat the process until convergence.

## Applications

The conjugate gradient method is often used as it's more efficient that other iterative solvers, such as steepest descent, and direct solvers, such as Gaussian Elimination. Iterative linear solvers are commonly used in optimisation, machine learning and computational fluid dynamics.

In [ ]:
import numpy as np

def conjugate_gradient(A: np.array, b: np.array, n: int, x0: np.array=None, tol=1e-8) -> np.array:
    """
    Solve the system Ax = b using the Conjugate Gradient method.

    :param A: Symmetric positive-definite matrix
    :param b: Right-hand side vector
    :param n: Maximum number of iterations
    :param x0: Initial guess for solution (default is zero vector)
    :param tol: Convergence tolerance
    :return: Solution vector x
    """

    # calculate initial residual vector
    x = np.zeros_like(b)
    r = residual(A, b, x) # residual vector
    rPlus1 = r
    p = r # search direction vector

    for i in range(n):

        # line search step value - this minimizes the error along the current search direction
        alp = alpha(A, r, p)

        # new x and r based on current p (the search direction vector)
        x = x + alp * p
        rPlus1 = r - alp * (A @ p)

        # calculate beta - this ensures that all vectors are A-orthogonal to each other
        bet = beta(r, rPlus1)

        # update x and r
        # using a othogonal search direction ensures we get all the information we need in more direction and then don't have to search in that direction again
        p = rPlus1 + bet * p

        # update residual vector
        r = rPlus1

        # break if less than tolerance
        if np.linalg.norm(residual(A, b, x)) < tol:
            break

    return x

def residual(A: np.array, b: np.array, x: np.array) -> np.array:
    # calculate linear system residuals
    return b - A @ x

def alpha(A: np.array, r: np.array, p: np.array) -> float:

    # calculate step size
    alpha_num = np.dot(r, r)
    alpha_den = np.dot(p @ A, p)

    return alpha_num/alpha_den

def beta(r: np.array, r_plus1: np.array) -> float:

    # calculate direction scaling
    beta_num = np.dot(r_plus1, r_plus1)
    beta_den = np.dot(r, r)

    return beta_num/beta_den


In [3]:
import numpy as np
A = np.array([[4, 1], [1, 3]])
b = np.array([1, 2])
n = 5
output = conjugate_gradient(A, b, n)
print('Test Case 1: Accepted') if np.allclose(output, [0.09090909, 0.63636364]) else print('Test Case 1: Failed')
print('Input:')
print('import numpy as np\nA = np.array([[4, 1], [1, 3]])\nb = np.array([1, 2])\nn = 5\nprint(conjugate_gradient(A, b, n))')
print()
print('Output:')
print(output)
print()
print('Expected:')
print('[0.09090909, 0.63636364]')
print()
print()
print()

import numpy as np
A = np.array([[4, 1, 2], [1, 3, 0], [2, 0, 5]])
b = np.array([7, 8, 5])
n = 1
output = conjugate_gradient(A, b, n)
print('Test Case 2: Accepted') if np.allclose(output, [1.2627451, 1.44313725, 0.90196078]) else print('Test Case 2: Failed')
print('Input:')
print('import numpy as np\nA = np.array([[4, 1, 2], [1, 3, 0], [2, 0, 5]])\nb = np.array([7, 8, 5])\nn = 1\nprint(conjugate_gradient(A, b, n))')
print()
print('Output:')
print(output)
print()
print('Expected:')
print('[1.2627451, 1.44313725, 0.90196078]')
print()
print()
print()

import numpy as np
A = np.array([[6, 2, 1, 1, 0],
              [2, 5, 2, 1, 1],
              [1, 2, 6, 1, 2],
              [1, 1, 1, 7, 1],
              [0, 1, 2, 1, 8]])
b = np.array([1, 2, 3, 4, 5])
n = 100
output = conjugate_gradient(A, b, n)
print('Test Case 3: Accepted') if np.allclose(output, [0.01666667, 0.11666667, 0.21666667, 0.45, 0.5]) else print('Test Case 3: Failed')
print('Input:')
print('import numpy as np\nA = np.array([[6, 2, 1, 1, 0],\n              [2, 5, 2, 1, 1],\n              [1, 2, 6, 1, 2],\n              [1, 1, 1, 7, 1],\n              [0, 1, 2, 1, 8]])\nb = np.array([1, 2, 3, 4, 5])\nn = 100\nprint(conjugate_gradient(A, b, n))')
print()
print('Output:')
print(output)
print()
print('Expected:')
print('[0.01666667, 0.11666667, 0.21666667, 0.45, 0.5]')

Test Case 1: Accepted
Input:
import numpy as np
A = np.array([[4, 1], [1, 3]])
b = np.array([1, 2])
n = 5
print(conjugate_gradient(A, b, n))

Output:
[0.09090909 0.63636364]

Expected:
[0.09090909, 0.63636364]



Test Case 2: Accepted
Input:
import numpy as np
A = np.array([[4, 1, 2], [1, 3, 0], [2, 0, 5]])
b = np.array([7, 8, 5])
n = 1
print(conjugate_gradient(A, b, n))

Output:
[1.2627451  1.44313725 0.90196078]

Expected:
[1.2627451, 1.44313725, 0.90196078]



Test Case 3: Accepted
Input:
import numpy as np
A = np.array([[6, 2, 1, 1, 0],
              [2, 5, 2, 1, 1],
              [1, 2, 6, 1, 2],
              [1, 1, 1, 7, 1],
              [0, 1, 2, 1, 8]])
b = np.array([1, 2, 3, 4, 5])
n = 100
print(conjugate_gradient(A, b, n))

Output:
[0.01666667 0.11666667 0.21666667 0.45       0.5       ]

Expected:
[0.01666667, 0.11666667, 0.21666667, 0.45, 0.5]
